In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from detect_outlier import *
from sklearn.model_selection import train_test_split


In [2]:
df = read_data_no_outlier()

In [3]:
def score(label):
    if label == 0:
        return np.random.randint(300, 580)
    if label == 1:
        return np.random.randint(580, 670)
    if label == 2:
        return np.random.randint(670, 740)
    if label == 3:
        return np.random.randint(740, 800)
    if label == 4:
        return np.random.randint(800, 850)
def labeling(scores):
    label = []
    for score in scores:
        if  score < 580:
            label.append(0)
        elif score >=580 and score < 670:
            label.append(1)
        elif score >= 670 and score < 740:
            label.append(2)
        elif score >= 740 and score < 800:
            label.append(3)
        elif score >= 800:
            label.append(4)
    return np.array(label)

In [4]:
df_normalized =df.drop("address", axis= 1).drop("createdAt", axis= 1).drop("zscore", axis= 1)
label_column = df['label']
df_normalized = (df_normalized - df_normalized.min()) / (df_normalized.max() - df_normalized.min())
df_normalized['label'] = label_column


In [5]:
X = np.array(df_normalized[
    [
        "balanceInUSD",
        # "borrowInUSD",
        # "depositInUSD",
        "averageTotalAsset",
        "frequencyMountOfTransaction",
        "borrow_per_balance",
        "deposit_per_balance",
        "borrow_per_deposit",
        "totalValueOfLiquidation",
        "numberOfLiquidation",
        "frequencyOfTransaction",
        "frequencyOfDappTransactions",
        "numberOfInteractedDapps",
        "typesOfInteractedDapps",
        "numberOfReputableDapps",
        "age",
    ]
].values)
y = df_normalized["label"].values
new_y = np.array([score(y[i]) for i in range(len(y))])



In [6]:
X_train, X_test, new_y_train, new_y_test, y_train, y_test  = train_test_split(
            X, new_y, y, test_size=0.2, random_state=41
        )
# Chuẩn bị dữ liệu


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(14,)),  # input_shape là (16,)
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [8]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, new_y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
2484/2484 [==============================] - 4s 1ms/step - loss: 42339.9844 - mae: 106.4495 - val_loss: 1669.3667 - val_mae: 30.2473
Epoch 2/10
2484/2484 [==============================] - 4s 2ms/step - loss: 1624.9037 - mae: 30.1310 - val_loss: 1639.6251 - val_mae: 30.0947
Epoch 3/10
2484/2484 [==============================] - 3s 1ms/step - loss: 1588.6006 - mae: 30.0048 - val_loss: 1599.8960 - val_mae: 29.9895
Epoch 4/10
2484/2484 [==============================] - 3s 1ms/step - loss: 1556.6572 - mae: 29.8500 - val_loss: 1569.1078 - val_mae: 29.7652
Epoch 5/10
2484/2484 [==============================] - 3s 1ms/step - loss: 1524.0271 - mae: 29.6846 - val_loss: 1539.9388 - val_mae: 29.6006
Epoch 6/10
2484/2484 [==============================] - 3s 1ms/step - loss: 1499.2026 - mae: 29.5380 - val_loss: 1544.1257 - val_mae: 30.0697
Epoch 7/10
2484/2484 [==============================] - 3s 1ms/step - loss: 1479.6759 - mae: 29.4165 - val_loss: 1495.6595 - val_mae: 29.3812
Epoc

In [9]:
X_test[2]

array([3.02416321e-06, 8.72215501e-06, 0.00000000e+00, 0.00000000e+00,
       5.00667484e-18, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.30911626e-01])

In [10]:
# X_new = np.expand_dims(X_test, axis=0)
predictions = model.predict(X_test)
print("Dự đoán:", predictions)
accuracy_score(y_test, labeling(predictions))

777/777 [==============================] - 1s 996us/step
Dự đoán: [[637.2034 ]
 [729.5453 ]
 [633.9678 ]
 ...
 [633.88293]
 [633.8444 ]
 [635.99146]]


0.8878376746245823